## TWO-STAGE CLUSTER ANALYSIS: EARLY PREPAYMENTS CUSTOMER SEGMENTATION

##### Import Libraries

In [1]:

# data handling
import pandas as pd
import numpy as np
from datetime import datetime
# data visualization
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.ticker import FuncFormatter
import seaborn as sns
# data science
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist
from scipy.stats import chi2_contingency

# to augment data
import random

#ignore wa
import warnings
warnings.filterwarnings('ignore')

#remove scientific notation
pd.options.display.float_format = '{:.2f}'.format

##### Import Data

In [2]:
raw_df = pd.read_csv("AUTO_PREPAYMENT_DATA.csv")

In [3]:
raw_df = raw_df.set_index('CUSTOMER_ID')

In [4]:
# augment data
sample_size = int(len(raw_df)*.5)
random_sample_indices = random.sample(list(raw_df.index), sample_size)
sample_df = raw_df.loc[random_sample_indices].copy()

raw_augmented_df = pd.concat([raw_df, sample_df], ignore_index=False)

In [5]:
raw_augmented_df = raw_augmented_df.reset_index()

##### Transform Data

In [6]:
today = pd.Timestamp('3/31/2025') 

In [7]:
# Convert to dates for calculation
raw_df['LOAN_OPEN_DATE'] = pd.to_datetime(raw_df['LOAN_OPEN_DATE'])
raw_df['LOAN_END_DATE'] = pd.to_datetime(raw_df['LOAN_END_DATE'])

# Add derived fields for cohorts
raw_df['OPEN_YEAR'] = raw_df['LOAN_OPEN_DATE'].dt.year
raw_df['OPEN_QTR'] = raw_df['LOAN_OPEN_DATE'].dt.quarter
raw_df['OPEN_YRQTR'] = raw_df['OPEN_YEAR'].astype(str) + 'Q' + raw_df['OPEN_QTR'].astype(str)

In [8]:
# Calculate Payoff Periods
raw_df['DAYS_TO_PAYOFF'] = (pd.to_datetime(raw_df['PAYOFF_DATE'])-pd.to_datetime(raw_df['LOAN_OPEN_DATE'])).dt.days
raw_df['MONTHS_TO_PAYOFF'] = round(((raw_df['DAYS_TO_PAYOFF']/365)*12))
raw_df['MONTHS_TO_PAYOFF'].loc[(raw_df['IS_PAID_OFF'] == 'YES') & (raw_df['MONTHS_TO_PAYOFF'] == 0.0)] = 1

In [9]:
PRE_FED = ['2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2021Q2','2021Q3', '2021Q4', '2022Q1', '2022Q2']
POST_FED = ['2022Q3', '2022Q4', '2023Q1','2023Q2', '2023Q3','2023Q4','2024Q1','2024Q2','2024Q3','2024Q4','2025Q1','2025Q2']

In [10]:
## Calculate Percentage of loan paid off
raw_df['PERCENT_COMPLETED'] = ((today - raw_df['LOAN_OPEN_DATE']).dt.days)/((raw_df['LOAN_END_DATE']-raw_df['LOAN_OPEN_DATE']).dt.days)*100

raw_df['PERCENT_COMPLETED'].loc[(raw_df['IS_PAID_OFF'] == 'YES') ] = 1